In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from qiime2 import Artifact

from skbio.diversity import beta_diversity, alpha_diversity
from skbio.stats import distance, ordination

from scipy.cluster import hierarchy
from scipy.spatial import distance

%matplotlib inline

# Read in Data

In [4]:
metadata = pd.read_csv("../metadata/mapping.tsv", sep="\t")

metabo_metadata = metadata.set_index("MetaboSampleID")



## Aqueous

In [5]:
aqueous = pd.read_csv("../metabolome/Aqueous_Green_Tea_compounds_in_Plasma_020722.txt",
                      sep="\t", header=11, index_col="Compound").T

raw_indexes        = [x for x in list(aqueous.index) if ("Log2(normalized)" not in x)]
normalized_indexes = [x for x in list(aqueous.index) if ("(raw)" not in x)]

raw_aqueous = aqueous.loc[raw_indexes]
log_aqueous = aqueous.loc[normalized_indexes]

raw_aqueous.index = [int(x.split("SBAQ_")[-1][:-5]) if ("SBAQ_" in x)
                     else x 
                     for x in list(raw_aqueous.index)]

log_aqueous.index = [int(x.split("SBAQ_")[-1][:-18]) if ("SBAQ_" in x)
                     else x 
                     for x in list(log_aqueous.index)]

metabo_info_cols = ['Compound Name','Retention Time', 'Measured Retention Time', 
                    'Product Mass','QualifierMass', 'CASNumber', 
                    'CompoundType', 'ISTDCompoundName', 'Mass Column']

raw_aqueous_samples = raw_aqueous.drop(metabo_info_cols).sort_index()
log_aqueous_samples = log_aqueous.drop(metabo_info_cols).sort_index()

## Lipid

In [6]:
lipid = pd.read_csv("../metabolome/Lipid_Green_Tea_compounds_in_Plasma_020722.txt",
                      sep="\t", header=11, index_col="Compound").T
lipid.head()

raw_lipid_indexes =        [x for x in list(lipid.index) if ("Log2(normalized)" not in x)]
normalized_lipid_indexes = [x for x in list(lipid.index) if ("(raw)" not in x)]

raw_lipid = lipid.loc[raw_lipid_indexes]
log_lipid = lipid.loc[normalized_lipid_indexes]

raw_lipid.index = [int(x.split("Lipid_")[-1][:-5]) if ("Lipid_" in x)
                     else x 
                     for x in list(raw_lipid.index)]

log_lipid.index = [int(x.split("Lipid_")[-1][:-18]) if ("Lipid" in x)
                     else x 
                     for x in list(log_lipid.index)]

raw_lipid_samples = raw_lipid.drop(metabo_info_cols).sort_index()
log_lipid_samples = log_lipid.drop(metabo_info_cols).sort_index()

## Microbiome

In [81]:
microbiome = Artifact.load("../microbiome/tax_filtered_GT.qza").view(pd.DataFrame)

tax = Artifact.load("../microbiome/taxonomy_GT.qza").view(pd.DataFrame)
tax = tax.to_dict()["Taxon"]
microbiome.rename(columns=tax, inplace=True) # map OTUs to taxonomy name

microbiome_mat = np.matrix(microbiome)
microbiome_mat /= microbiome_mat.sum(axis=1) #convert to relative abundances

microbiome = pd.DataFrame(microbiome_mat, columns=microbiome.columns, index=microbiome.index)

In [82]:
microbiome_index_mapper = metadata.set_index("#SampleID")["MetaboSampleID"].to_dict()

new_microbiome_index = [int(microbiome_index_mapper[x]) for x in list(microbiome.index)]
microbiome.index = new_microbiome_index
microbiome = microbiome.sort_index()

# Presence/Absence correlation

## Make presence/absence

In [84]:
def make_presence_absence(x, threshold):
    if x > threshold:
        return 1
    else:
        return 0

In [85]:
presence_aqueous_samples = pd.DataFrame(index = raw_aqueous_samples.index)
presence_lipid_samples = pd.DataFrame(index = raw_lipid_samples.index)
presence_microbiome = pd.DataFrame(index = microbiome.index, columns=microbiome.columns)

for col in raw_aqueous_samples.columns:
    presence_aqueous_samples[col] = raw_aqueous_samples[col].apply(lambda x: make_presence_absence(x, threshold=0.00001))
    
for col in raw_lipid_samples.columns:
    presence_lipid_samples[col] = raw_lipid_samples[col].apply(lambda x: make_presence_absence(x, threshold=0.00001))

for col in microbiome.columns:
    presence_microbiome[col] = microbiome[col].apply(lambda x: -1 * (-x//1)) #ceiling division to convert to presence


## Correlation

In [70]:
from scipy.stats import spearmanr

In [87]:
all_presence = pd.concat([presence_microbiome, presence_aqueous_samples, presence_lipid_samples], axis=1)

In [99]:
all_presence_corr = np.matrix(all_presence.corr(method="spearman"))


## Network

In [92]:
import networkx as nx

In [100]:
G = nx.from_numpy_matrix(all_presence_corr)